<!-- Copyright (c) 2021-2022, InterDigital Communications, Inc
All rights reserved.

Redistribution and use in source and binary forms, with or without 
modification, are permitted (subject to the limitations in the disclaimer 
below) provided that the following conditions are met:

* Redistributions of source code must retain the above copyright notice, 
this list of conditions and the following disclaimer.
* Redistributions in binary form must reproduce the above copyright notice, 
this list of conditions and the following disclaimer in the documentation 
and/or other materials provided with the distribution.
* Neither the name of InterDigital Communications, Inc nor the names of its 
contributors may be used to endorse or promote products derived from this 
software without specific prior written permission.

NO EXPRESS OR IMPLIED LICENSES TO ANY PARTY'S PATENT RIGHTS ARE GRANTED BY 
THIS LICENSE. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND 
CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT 
NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A 
PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER 
OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, 
EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, 
PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; 
OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, 
WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR 
OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF 
ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. -->

# Collect results

## 1 - Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '9'

In [ ]:
import math
import io
import torch
from torchvision import transforms
import numpy as np

from PIL import Image
from functools import partial
import matplotlib.pyplot as plt
from pytorch_msssim import ms_ssim
from compressai.zoo import bmshj2018_factorized
from ipywidgets import interact, widgets
import pandas as pd

from dotmap import DotMap
import numpy as np
import toml
import sys
sys.path.insert(1, "licos")
from licos.raw_image_folder import RawImageFolder
from torchvision.transforms.functional import rotate

device = "cuda" if torch.cuda.is_available() else "cpu"
from torchvision import transforms
from tqdm import tqdm

from eval_utils import (
    compute_msssim,
    compute_psnr,
    pillow_encode,
)

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from licos.model_utils import get_model
from skimage.util import img_as_ubyte
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.patches import Rectangle


## 2 - Collect and plot results

Collect merged.

In [ ]:
q1 = pd.read_csv("/home/gabrielemeoni/project/LICOS/results_log/results_merged_seed_2_qual_1.csv")
q2 = pd.read_csv("/home/gabrielemeoni/project/LICOS/results_log/results_merged_seed_2_qual_2.csv")
q4 = pd.read_csv("/home/gabrielemeoni/project/LICOS/results_log/results_merged_seed_2_qual_4.csv")
q8 = pd.read_csv("/home/gabrielemeoni/project/LICOS/results_log/results_merged_seed_2_qual_8.csv")

In [ ]:
bpp_merged = [q1["BPP"], q2["BPP"], q4["BPP"], q8["BPP"]]
ssim_merged = [q1["SSIM"], q2["SSIM"], q4["SSIM"], q8["SSIM"]]
psnr_merged = [q1["PSNR"], q2["PSNR"], q4["PSNR"], q8["PSNR"]]
bpp_merged = np.array([float(x) for x in bpp_merged])
ssim_merged = np.array([float(x) for x in ssim_merged])
psnr_merged = np.array([float(x) for x in psnr_merged])

Collect split.

In [ ]:
q1 = pd.read_csv("/home/gabrielemeoni/project/LICOS/results_log/results_split_seed_2_qual_1.csv")
q2 = pd.read_csv("/home/gabrielemeoni/project/LICOS/results_log/results_split_seed_2_qual_2.csv")
q4 = pd.read_csv("/home/gabrielemeoni/project/LICOS/results_log/results_split_seed_2_qual_4.csv")
q8 = pd.read_csv("/home/gabrielemeoni/project/LICOS/results_log/results_split_seed_2_qual_8.csv")
bpp_split = [q1["BPP"], q2["BPP"], q4["BPP"], q8["BPP"]]
ssim_split = [q1["SSIM"], q2["SSIM"], q4["SSIM"], q8["SSIM"]]
psnr_split = [q1["PSNR"], q2["PSNR"], q4["PSNR"], q8["PSNR"]]
bpp_split = np.array([float(x) for x in bpp_split])
ssim_split = np.array([float(x) for x in ssim_split])
psnr_split = np.array([float(x) for x in psnr_split])

Collect JPEG.

In [ ]:
jpeg_df = pd.read_csv("/home/gabrielemeoni/project/LICOS/results_log/results_jpeg_seed_2.csv")
bpp_jpeg = np.array(jpeg_df["BPP"])
psnr_jpeg = np.array(jpeg_df["PSNR"])
ssim_jpeg = np.array(jpeg_df["SSIM"])
bpp_jpeg = np.array([float(x) for x in bpp_jpeg])
ssim_jpeg = np.array([float(x) for x in ssim_jpeg])
psnr_jpeg = np.array([float(x) for x in psnr_jpeg])


Plot results.

In [ ]:
##### plt.rcParams.update({'font.size': 10})
plt.close()
fig, ax = plt.subplots(1,2, figsize=(6, 3))
ax[0].scatter(sorted(bpp_jpeg), ssim_jpeg[np.argsort(bpp_jpeg)], color="b", marker="x")
ax[0].scatter(sorted(bpp_merged), ssim_merged[np.argsort(bpp_merged)], color="r", marker="x")
ax[0].scatter(sorted(bpp_split), ssim_split[np.argsort(bpp_split)], color="g", marker="x")
ax[0].legend(["JPEG", "MERGED", "SPLIT"])
ax[0].set_xlabel("BPP")
ax[0].set_ylabel("SSIM")
ax[1].scatter(sorted(bpp_jpeg), psnr_jpeg[np.argsort(bpp_jpeg)], color="b", marker="x")
ax[1].scatter(sorted(bpp_merged), psnr_merged[np.argsort(bpp_merged)], color="r", marker="x")
ax[1].scatter(sorted(bpp_split), psnr_split[np.argsort(bpp_split)], color="g", marker="x")
ax[1].legend(["JPEG", "MERGED", "SPLIT"])
ax[1].set_xlabel("BPP")
ax[1].set_ylabel("PSNR")
ax[0].set_xlim(0, 0.125)
ax[0].set_ylim(0.5, 1.0)
ax[1].set_xlim(0, 0.125)
ax[1].set_ylim(27, 45)

#plt.subplots_adjust(wspace=0.4, 
                    #hspace=3)
plt.tight_layout() 
plt.savefig("results.pdf")

## 3 - Create compression and decompression results

Adjust path t models checkpoints

In [ ]:
# Path to models
paths=['/home/gabrielemeoni/project/LICOS/licos/results/bmshj2018-factorized_qual=1_raw=split_seed=2_t=2024_03_07_08_44_51.pth.tar', "/home/gabrielemeoni/project/LICOS/licos/results/bmshj2018-factorized_qual=4_raw=split_seed=2_t=2024_03_08_06_39_18.pth.tar"]

Running split and decompressed

In [ ]:
{"JPG": 0, "QUAL_2": 0, "QUAL_8": 0}

In [ ]:
# Tranform useful for JPEG
transform = transforms.Compose([transforms.PILToTensor()])
# Placeholder for images
orig_img = []
jpg_img = []
qual_2_img = []
qual_8_img = []
# Lit of images to collect
idx_list = [44, 93, 555, 1000]

# Compression ratio dicitionary
compression_ratios = dict(zip([str(n) for n in idx_list], [{"JPG": 0, "QUAL_2": 0, "QUAL_8": 0} for n in idx_list]))

# Run

for n in tqdm([44, 93, 555, 1000], "parse idx.."):
    for path in paths:

        qual = int(path.split("_qual=")[1].split("_raw")[0])
        seed = int(path.split("_seed=")[1].split("_t")[0])
        raw_model = str(path.split("_raw=")[1].split("_seed")[0])
        cfg_path = "/home/gabrielemeoni/project/LICOS/cfg/raw_" + raw_model + "_seed_" + str(seed) + "_q_" + str(qual) +".toml"
        cfg = DotMap(toml.load(cfg_path), _dynamic=False)
        test_data_split = RawImageFolder(
                    cfg.dataset,
                    cfg.seed,
                    cfg.raw_test_over_tot,
                    cfg.raw_validation_over_train,
                    "split",
                    cfg.raw_target_resolution_merged_m,
                    split="test",
                    geographical_split_tolerance=cfg.raw_train_test_tolerance,
                )
        net_split = get_model("bmshj2018-factorized", False, 1, qual)
        checkpoint = torch.load(path, map_location=device)
        net_split.load_state_dict(checkpoint["state_dict"])
        net_split.update()
        img = test_data_split[n]
        x = img.to(device)
        net_split=net_split.cuda()
        with torch.no_grad():
            out_net = net_split.forward(x.unsqueeze(0))
            compressed_img = net_split.compress(x.unsqueeze(0))
            compressed_size_in_bytes = np.frombuffer(
                np.array(compressed_img["strings"]), dtype=np.uint8
                ).size
            img_out_b = out_net["x_hat"].clamp_(0, 1)
        img_size_in_bytes = img.shape[0] * img.shape[1] * img.shape[2]
        
        bpp = 8 * compressed_size_in_bytes/img_size_in_bytes

        if qual == 1:
            qual_2_img.append(img_as_ubyte(img_out_b.squeeze(0).squeeze(0).detach().cpu().numpy()))
            compression_ratios[str(n)]["QUAL_2"] = bpp/8
        else:
            qual_8_img.append(img_as_ubyte(img_out_b.squeeze(0).squeeze(0).detach().cpu().numpy()))
            compression_ratios[str(n)]["QUAL_8"] = bpp/8
    # JPEG

    PIL_img = Image.fromarray(img_as_ubyte(img.squeeze(0)), "L")
    img_jpeg_0, bpp_jpeg_img = pillow_encode(PIL_img, fmt="jpeg", quality=8)
    img_jpeg_0=transform(img_jpeg_0)
    img_jpeg = img_jpeg_0
    compression_ratios[str(n)]["JPEG"] = bpp_jpeg_img / 8
    jpg_img.append(img_jpeg.squeeze(0).detach().cpu().numpy())
    orig_img.append(img_as_ubyte(img.squeeze(0).detach().cpu().numpy()))
    

# Create dataframe collecting results
df = pd.DataFrame(columns=["ImageIDX","SSIM_JPG","PSNR_JPG","SSIM_QUAL_2","PSNR_QUAL_2","SSIM_QUAL_8","PSNR_QUAL_8"])
df.set_index("ImageIDX")
for i in range(len(idx_list)):
    data_dict = {
        "ImageIDX": idx_list[i],
        "SSIM_JPG": ssim(orig_img[i],jpg_img[i],data_range=255,multichannel=True),
        "PSNR_JPG": psnr(orig_img[i],jpg_img[i],data_range=255),
        "SSIM_QUAL_2": ssim(orig_img[i],qual_2_img[i],data_range=255,multichannel=True),
        "PSNR_QUAL_2": psnr(orig_img[i],qual_2_img[i],data_range=255),
        "SSIM_QUAL_8": ssim(orig_img[i],qual_8_img[i],data_range=255,multichannel=True),
        "PSNR_QUAL_8": psnr(orig_img[i],qual_8_img[i],data_range=255),
        "CR_JPG": compression_ratios[str(idx_list[i])]["JPG"],
        "CR_QUAL_2": compression_ratios[str(idx_list[i])]["QUAL_2"],
        "CR_QUAL_8": compression_ratios[str(idx_list[i])]["QUAL_8"]
    }
    new_df = pd.DataFrame(data_dict,index=[0])
    new_df.set_index("ImageIDX")
    df = pd.concat([df,new_df])

Plot results.

In [ ]:

def make_zoom_in(ax, axins, orig_img, region):
    ax.imshow(orig_img, cmap='gray')
    ax.axis('off')

    # Specify the region to zoom in on
    x1, x2, y1, y2 = region
    zoomed_img = orig_img[y1:y2, x1:x2]

    # Flip the zoomed image in the up-down direction
    flipped_zoomed_img = np.flipud(zoomed_img)

    axins.imshow(flipped_zoomed_img, cmap='gray')

    # Set the limits of the inset axes to the full extent of the zoomed image
    axins.set_xlim(0, zoomed_img.shape[1])  # apply the x-limits
    axins.set_ylim(0, zoomed_img.shape[0])  # apply the y-limits

    # Draw a rectangle on the original axes to indicate the zoomed area
    rect = Rectangle((x1, y1), x2-x1, y2-y1, fill=False, color='red', linestyle='dashed')
    ax.add_patch(rect)

    axins.axis('off')
    axins.indicate_inset_zoom(axins, edgecolor="red", linestyle='dashed', lw=4)

    return ax, axins

size = 150
x0 = 10
y0 = 10
x1 = 1000
y1 = 400
x2 = 400
y2 = 475
x3 = 1100
y3 = 1000
zoom_in_regions = [
    (x0,x0+size,y0,y0+size),
    (x1,x1+size,y1,y1+size),
    (x2,x2+size,y2,y2+size),
    (x3,x3+size,y3,y3+size)
]                    


aspect_ratio = 1296/1152
fig, axs = plt.subplots(len(idx_list), 4, figsize=(20,20/aspect_ratio), sharex=True, sharey=True)
# Put column titles
axs[0,0].set_title("Original")
axs[0,1].set_title("JPEG (q=8)")
axs[0,2].set_title("Split (q=1)")
axs[0,3].set_title("Split (q=4)")
for i in range(len(idx_list)):
    axs[i,0],axs[i,0] = make_zoom_in(axs[i,0],inset_axes(axs[i,0], width="40%", height="40%", loc=3),orig_img[i],zoom_in_regions[i])
    axs[i,1].annotate("SSIM: "+str(round(df.iloc[i]["SSIM_JPG"],2))+"\nPSNR: "+str(round(df.iloc[i]["PSNR_JPG"],2))+"\nCR: "+str(round(df.iloc[i]["CR_JPG"],2)), xy=(0,0.), xycoords='axes fraction',xytext=(0.65,0.8), fontsize=16, color='red')
    axs[i,1],axs[i,1] = make_zoom_in(axs[i,1],inset_axes(axs[i,1], width="40%", height="40%", loc=3),jpg_img[i],zoom_in_regions[i])
    axs[i,2].annotate("SSIM: "+str(round(df.iloc[i]["SSIM_QUAL_2"],2))+"\nPSNR: "+str(round(df.iloc[i]["PSNR_QUAL_2"],2))+"\nCR: "+str(round(df.iloc[i]["CR_QUAL_2"],2)), xy=(0,0.), xycoords='axes fraction',xytext=(0.65,0.8), fontsize=16, color='red')
    axs[i,2],axs[i,2] = make_zoom_in(axs[i,2],inset_axes(axs[i,2], width="40%", height="40%", loc=3),qual_2_img[i],zoom_in_regions[i])
    axs[i,3].annotate("SSIM: "+str(round(df.iloc[i]["SSIM_QUAL_8"],2))+"\nPSNR: "+str(round(df.iloc[i]["PSNR_QUAL_8"],2))+"\nCR: "+str(round(df.iloc[i]["CR_QUAL_8"],2)), xy=(0,0.), xycoords='axes fraction',xytext=(0.65,0.8), fontsize=16, color='red')
    axs[i,3],axs[i,3] = make_zoom_in(axs[i,3],inset_axes(axs[i,3], width="40%", height="40%", loc=3),qual_8_img[i],zoom_in_regions[i])

plt.rcParams.update({'font.size': 32})
plt.subplots_adjust(wspace=0, hspace=0)
plt.savefig("comparison.pdf")